In [1]:
from flask import Flask, render_template, request, jsonify
import re 
import ast
from flask_cors import CORS
import pandas as pd

app = Flask(__name__)
CORS(app)


In [2]:
from groq import Groq

client = Groq(
    api_key="your_api_key",
)
df = []
tst = ""
info = ""

In [3]:
def search_dataset(message) :
    # Search DATASET ...,

    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": f"{message} , Provide me top 5 links from kaggle to download the dataset , be precise" ,
        }
    ],
    model="llama3-70b-8192",
    )
    response_text = chat_completion.choices[0].message.content
    print(response_text)
    return response_text


In [4]:
def preprocess(mess) :

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"This is the dataset i have : {tst} , give a code to {mess} also call it , give code only dont give explanation , do all in inplace so that it updates in realtime" ,
            }
        ],
        model="llama3-70b-8192",
        )
    
    akn = client.chat.completions.create(
        messages=[
            {
        "role": "user",
        "content": "Write a success aknowledgement message in 3 lines for {mess}" ,
    }
        ],
        model="llama3-70b-8192",
        )
    viss = chat_completion.choices[0].message.content
    viss =  re.sub('python', '', viss)
    viss =  re.sub('Python', '', viss)
    
    print(viss)
    pattern = r"```(.*?)```"

    # Find all matches
    matches = re.findall(pattern, viss, re.DOTALL)

    # Safely parse the response text as a Python AST (Abstract Syntax Tree)
    parsed_ast = ast.parse(matches[0])

    # else:
    # Execute the parsed AST
    exec(compile(parsed_ast, filename="<ast>", mode="exec"))
    
    return akn.choices[0].message.content


In [5]:
@app.route('/process_file', methods=['POST'])
def process_file():
    global info 
    global df
    global tst 
    file = request.files['file']
    if file:
        file.save('data.csv')
        df = pd.read_csv('data.csv')
        tst = df.head()
        info = df.info()
        chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"This is the dataset i have : {tst} , explain and summarize it in 2 lines" ,
            }
        ],
        model="llama3-70b-8192",
        )
        response_text = chat_completion.choices[0].message.content
        print(response_text)
        return jsonify({'response': response_text})
    else:
        return jsonify({'response': 'No file provided'})




In [6]:
# # # Process CSV 
# # @app.route('/process_data', methods=['POST'])

# def process_csv () :
#     global df
#     path = ""
#     df = pd.read_csv(path)
#     chat_completion = client.chat.completions.create(
#     messages=[
#         {
#             "role": "user",
#             "content": f"This is the dataset i have : {tst} , explain and summarize it in 2 lines" ,
#         }
#     ],
#     model="llama3-8b-8192",
#     )
#     response_text = chat_completion.choices[0].message.content
#     print(response_text)


In [7]:
def suggest() :
    # Suggessions

    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": f"I'm working on preprocessing a DataSet ,  This is the dataset i have : {tst} and its info : {info},give me 3 suggests to what to do further in 3 lines" ,
        }
    ],
    model="llama3-70b-8192",
    )
    response_text = chat_completion.choices[0].message.content
    return response_text

In [8]:
df

[]

In [9]:
cnt=1

In [10]:
def visualize(mess) :
    # Visualize 
    global cnt
    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            # "content": f"This is the dataset i have : {tst} , give a code to visualize column1 and column2 also call it ,mention the column names exactly , give code only dont give explanation " ,
            "content": f"This is the dataset i have : {tst} , give a code to visualize {mess}using matplotlib , also call it ,give the column names exactly as in the dataset give clean code also save the plot as 'plot{cnt}.png' "
        }
    ],
    model="llama3-70b-8192",
    )
    viss = chat_completion.choices[0].message.content
    viss =  re.sub('python', '', viss)
    viss =  re.sub('Python', '', viss)
    pattern = r"```(.*?)```"

    # Find all matches
    matches = re.findall(pattern, viss, re.DOTALL)

    # Safely parse the response text as a Python AST (Abstract Syntax Tree)
    parsed_ast = ast.parse(matches[0])

    # else:
    # Execute the parsed AST
    exec(compile(parsed_ast, filename="<ast>", mode="exec"))
    
    return f"C:/Prodigy/DataGenie/plot{cnt}.png"



In [11]:

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/send_message', methods=['POST'])
def send_message():
    global cnt
    message = request.get_json()['message']
    if "visualize" in message.lower() or "plot" in message.lower() :
        res = visualize(message)
        cnt += 1
        return jsonify({'response': "image" , "path" : res})
    response = process_message(message)
    return jsonify({'response': response })

def process_message(message):
    # Implement your chatbot logic here
    res = ""
    if "search a dataset" in message.lower() :
        res = search_dataset(message)
        # return res

        # return res
    if 'preprocess' in message.lower() or 'clean' in message.lower() :
        res = preprocess(message)
        # return res
    if 'suggest' in message.lower() or 'next' in message.lower() :
        res = suggest()
        # return res
    
    return res
    return "image"


,Alcohol value,Methane value,Temperature,Humidity,Food Condition,Life Expectancy,Estimated Ethylene Level
0,440.00,435.00,42.00,46.00,Spoiled,2.380952,100
1,370.00,380.00,35.00,40.00,Unspoiled,8.333333,29
2,405.00,410.00,40.00,43.00,Unspoiled,5.263158,14
3,345.00,355.00,32.00,39.00,Unspoiled,10.526316,42
4,430.00,420.00,41.00,45.00,Spoiled,2.857143,100
...,...,...,...,...,...,...,...
91,446.27,432.84,33.47,39.15,Spoiled,2.000000,100
92,398.51,364.92,37.71,43.58,Unspoiled,6.000000,28
93,311.73,443.28,42.49,48.92,Spoiled,2.000000,100
94,424.14,406.57,35.34,40.27,Unspoiled,7.000000,20


In [14]:
if __name__ == '__main__':
    app.run(port=7000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:7000
Press CTRL+C to quit
